In [1]:
import h5py as h5
import hdf5storage
import numpy as np
import imageio
import mat73
import os
import cv2
import random
import time
import argparse

In [2]:
def savePic(picture,fileName,extention,outPath): #saves the given array as a pictures to the given output path
    outPath = outPath+fileName+'.'+extention
    print(outPath)
    try:
        #old
        #imageio.imwrite(outPath,picture,format=extention)#extention'')#save the Data (path with name and file extention,PictureMatrix,format)
        #new
        cv2.imwrite(outPath,picture)
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(')
        print('--------------------')

In [3]:
def YUVtoRGB(img):
    print('moi')
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_YUV2RGB, cv2.IMREAD_UNCHANGED) #uses the CV2 method to convert the color space from YU-V to RGB
    print(pictureYUV)
    return pictureYUV

In [4]:
def convert(img, target_type_min, target_type_max): # converts the input array to a target type with the bounderys given
    imin = img.min()
    imax = img.max()
    #print(imin)
    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b)
    return new_img

In [17]:
#finished 24012021
# Reads a matlab File and saves the 
def recoverpic(pathToFile):
    mat = hdf5storage.loadmat(pathToFile)
    key = mat.keys()
    key = str(key).split('\'')[1]
    allPictures = int((mat[key][0,0,0,:]).shape[0]) # reads the amount of pictures in the .mat file
    outPath = input('Where should the data be saved?[default: /home/nico/rohdaten/reallive/png/ ]: ') or '/home/nico/rohdaten/reallive/png/'
    numberOfP = input('How many pictures from: '+str(allPictures)+' should be saved?[default: 23 ]: ') or '23'#str(allPictures-1)
    numberOfP = int(numberOfP)
    #fileName = input('what should be the name? default: outp_ ') or 'outp_'
    toPictures = allPictures - numberOfP
    #print(key)
    if (key == 'HDR_data'):
        print('HDR')
        while (toPictures <= int(allPictures)-1):
            picture = (mat[key][:,:,:,toPictures])#.astype(np.uint16) # create an array with the right size
            # conversion to 0,1??
            print('Writing HDR picture to: ')
            i = 0
            kaka = 'abc'+str(toPictures)
            savePic(picture,kaka,'hdr',outPath)
            picture_new = picture / ((2 ** 16)-1)
            picture_new = convert(picture_new,0,1)
            picture_new = picture_new.astype(np.float32)
            print(picture_new.max())
            print(picture_new.min())
            picture_new = YUVtoRGB(picture_new)
            picture_new = picture_new * ((2 ** 16)-1)
            kaka = 'abc'+str(toPictures)+'rgb'
            savePic(picture_new,kaka,'hdr',outPath)
            while(i<=2):
                if(i==1):
                    outName = str(toPictures)+'-u_pred'
                if(i==0):
                    outName = str(toPictures)+'-y_pred'
                if(i==2):
                    outName = str(toPictures)+'-v_pred'
                savePic((picture[:,:,i]),outName,'png',outPath) #it saves as a 16bit png not perfect RE DO?
                i = i + 1
            toPictures = toPictures+1
            
    if (key == 'SDR_data'):
        print('SDR')
        print(toPictures)
        print(allPictures)
        while (toPictures <= allPictures-1):
            picture = mat[key][:,:,:,toPictures].astype(np.uint8)
            i = 0
            kaka = 'abc'+str(toPictures)
            savePic(picture,kaka,'png',outPath)
            print('Writing PNG picture to: ')
            while(i<=2):
                if(i==2):
                    outName = str(toPictures)+'-v_pred'
                if(i==1):
                    outName = str(toPictures)+'-u_pred'
                if(i==0):
                    outName = str(toPictures)+'-y_pred'
                savePic((picture[:,:,i]),outName,'png',outPath)
                i = i + 1
            toPictures = toPictures+1
    print('finished writing Pictures to: ' + outPath)

In [18]:
start_time = time.time()
try:
    pathtomat = input('Please insert the path to your picture .mat File default: /home/nico/jsiGan/JSI-GAN/HDR_data.mat') or '/home/nico/jsiGan/JSI-GAN/HDR_data.mat'
    recoverpic(pathtomat)
    
except: 
    print('decoding .mat failed')
print(str((time.time() - start_time)/60)+'Minutes') #outputs the time in minutes
print('------------------------- Done --------------------')

Please insert the path to your picture .mat File default: /home/nico/jsiGan/JSI-GAN/HDR_data.mat
Where should the data be saved?[default: /home/nico/rohdaten/reallive/png/ ]: 
How many pictures from: 2550 should be saved?[default: 23 ]: 
HDR
Writing HDR picture to: 
/home/nico/rohdaten/reallive/png/abc2527.hdr
1.0
0.0
moi
[[[0.13584675 0.1529051  1.1998581 ]
  [0.1434611  0.16051945 1.2074724 ]
  [0.15109044 0.16053265 1.2074668 ]
  ...
  [0.10534226 0.09191634 1.0016732 ]
  [0.08246834 0.06142112 0.96356606]
  [0.05198003 0.02331759 0.91781366]]

 [[0.12821712 0.14527547 1.1922284 ]
  [0.1434611  0.16051945 1.2074724 ]
  [0.15109073 0.16814908 1.2151021 ]
  ...
  [0.1282162  0.12241153 1.0397803 ]
  [0.12821588 0.11478904 1.0321761 ]
  [0.12058595 0.09954387 1.0092819 ]]

 [[0.13584675 0.1529051  1.1998581 ]
  [0.15109073 0.16814816 1.2227314 ]
  [0.17396437 0.1910218  1.245605  ]
  ...
  [0.14347574 0.14529234 1.0702732 ]
  [0.15108983 0.14528424 1.0702832 ]
  [0.14346021 0.13765553 

/home/nico/rohdaten/reallive/png/2535-y_pred.png
/home/nico/rohdaten/reallive/png/2535-u_pred.png
/home/nico/rohdaten/reallive/png/2535-v_pred.png
Writing HDR picture to: 
/home/nico/rohdaten/reallive/png/abc2536.hdr
1.0
0.0
moi
[[[ 0.17628315 -0.06322308  0.31638774]
  [ 0.17309703 -0.06957     0.3096378 ]
  [ 0.17708884 -0.06520105  0.3132145 ]
  ...
  [ 0.16595843 -0.07829779  0.3279061 ]
  [ 0.16357003 -0.0814783   0.32471302]
  [ 0.15486494 -0.09298689  0.31243595]]

 [[ 0.1786752  -0.06043498  0.31918216]
  [ 0.17826994 -0.06322272  0.31639376]
  [ 0.1806595  -0.06163039  0.31678516]
  ...
  [ 0.16516066 -0.08028375  0.32590127]
  [ 0.16953824 -0.07472661  0.33305526]
  [ 0.1576284  -0.08941882  0.31639737]]

 [[ 0.1790609  -0.05885496  0.32074395]
  [ 0.18024534 -0.06084381  0.31797135]
  [ 0.19334938 -0.0473437   0.3314777 ]
  ...
  [ 0.16357827 -0.08187345  0.32511902]
  [ 0.16834527 -0.07630919  0.3318842 ]
  [ 0.15763298 -0.08941554  0.31717107]]

 ...

 [[ 0.36244485  0.378

/home/nico/rohdaten/reallive/png/2544-v_pred.png
Writing HDR picture to: 
/home/nico/rohdaten/reallive/png/abc2545.hdr
1.0
0.0
moi
[[[0.31222326 0.20833424 1.0056293 ]
  [0.30061236 0.1822335  0.9737106 ]
  [0.29193234 0.1619378  0.94183177]
  ...
  [0.26584074 0.1329418  0.92732584]
  [0.2629247  0.12714964 0.9186197 ]
  [0.24842407 0.10974185 0.89543855]]

 [[0.262936   0.13293691 0.9360367 ]
  [0.26293078 0.12424809 0.92443454]
  [0.26582515 0.11555418 0.9012434 ]
  ...
  [0.26004165 0.1300452  0.9273342 ]
  [0.26293567 0.13294053 0.92732424]
  [0.25133365 0.11265482 0.9041252 ]]

 [[0.2252334  0.0662486  0.8577494 ]
  [0.22523886 0.06335431 0.8461393 ]
  [0.23393255 0.05755338 0.8316523 ]
  ...
  [0.25712624 0.13003619 0.9215143 ]
  [0.265834   0.13874523 0.92731786]
  [0.26873899 0.13874649 0.9273194 ]]

 ...

 [[0.20206228 0.09812879 1.0954573 ]
  [0.20783962 0.09812494 1.0954452 ]
  [0.21943647 0.10972179 1.1070421 ]
  ...
  [0.30352956 0.14164074 0.9737057 ]
  [0.2832253  0.118